In [1]:
from scripts.analytics import Analytics
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

calculadora =  Analytics()

In [2]:
def print_report(dono:str):
    res = calculadora.calcular_patrimonio(dono=dono)
    cdi = calculadora.calcular_cdi_proventos(dono=dono)
    ativos = calculadora.calcular_valorizacao_ativos(dono=dono)
    proventos = calculadora.calcular_ativos_proventos(dono=dono)
    
    dolar_hoje = res['dolar_hoje']

    print(f"""
    -------- {dono.title()} Patrimonio --------
    
    Patrimonio total: R${res['patrimonio_total']:,.2f}

    Moeda Real: R${res['patrimonio_moeda_real']:,.2f}
    Ativos Real: R${res['patrimonio_ativos_real']:,.2f}
    Moeda Dolar: ${res['patrimonio_moeda_dolar']:,.2f} (R${res['patrimonio_moeda_dolar']*dolar_hoje:,.2f})
    Ativos Dolar: ${res['patrimonio_ativos_dolar']:,.2f} (R${res['patrimonio_ativos_dolar']*dolar_hoje:,.2f})

    ---------- Detalhamento ----------
    """)


    col_1 = []
    col_2 = []
    col_3 = []
    col_4 = []
    for key, caixa in cdi.items():
        
        if cdi[key]["valor"] <= 0:
            continue

        rendimento_p = (cdi[key]["rendimento"]/cdi[key]["valor"])*100

        col_1.append(key)
        col_2.append(f'R$ {cdi[key]["valor"]:,.2f}')
        col_3.append(f'% {rendimento_p:,.2f}')
        col_4.append(f'R$ {cdi[key]["rendimento"]:,.2f}')

    df_cdi = pd.DataFrame()
    df_cdi['caixa'] = col_1
    df_cdi['valor'] = col_2
    df_cdi['rendimento %'] = col_3
    df_cdi['rendimento'] = col_4

    print(df_cdi.to_string(index=False))
    print('')

    col_1 = []
    col_2 = []
    col_3 = []
    col_4 = []
    valorizacao_carteira = {'investido': 0, 'valor_atual': 0, 'proventotos': 0}
    for key, ativo in ativos.items():
        prov = 0
        if key in proventos:
            prov = proventos[key]['proventos']

        valor = ativo['price_today'] * ativo['quantidade']
        valorizacao = ativo['valorizacao'] * 100

        if key in ['MMM']:
            valorizacao_carteira['investido'] += ativo['valor_investido'] * dolar_hoje
            valorizacao_carteira['valor_atual'] += valor * dolar_hoje
            valorizacao_carteira['proventotos'] += prov * dolar_hoje
        else:
            valorizacao_carteira['investido'] += ativo['valor_investido']
            valorizacao_carteira['valor_atual'] += valor
            valorizacao_carteira['proventotos'] += prov

        col_1.append(key)
        col_2.append(f'R$ {valor:,.2f}')
        col_3.append(f'% {valorizacao:,.2f}')
        col_4.append(f'R$ {prov:,.2f}')

    df_ativos = pd.DataFrame()
    df_ativos['ativo'] = col_1
    df_ativos['valor'] = col_2
    df_ativos['valorização'] = col_3
    df_ativos['proventos'] = col_4

    print(df_ativos.to_string(index=False))

    ativos_valor_atual_mais_proventos = valorizacao_carteira['valor_atual'] + valorizacao_carteira['proventotos']
    ativos_valorizacao = ((valorizacao_carteira['valor_atual']/valorizacao_carteira['investido'])-1)*100
    ativos_valorizacao_mais_proventos = (((valorizacao_carteira['valor_atual']+valorizacao_carteira['proventotos'])/valorizacao_carteira['investido'])-1)*100
    print(f"""
    
    -------- {dono.title()} Carteira --------

    Investido: R${valorizacao_carteira['investido']:,.2f}

    Valor atual: R${valorizacao_carteira['valor_atual']:,.2f}
    Proventos: R${valorizacao_carteira['proventotos']:,.2f}
    Valor atual + Proventos: R${ativos_valor_atual_mais_proventos:,.2f}

    Valorização: %{ativos_valorizacao:,.2f}
    Valorização + Proventos: %{ativos_valorizacao_mais_proventos:,.2f}

    """)

    


In [4]:
print_report('rodrigo')


    -------- Rodrigo Patrimonio --------
    
    Patrimonio total: R$104,378.63

    Moeda Real: R$41,156.55
    Ativos Real: R$41,279.73
    Moeda Dolar: $2,467.94 (R$12,188.42)
    Ativos Dolar: $1,975.00 (R$9,753.93)

    ---------- Detalhamento ----------
    
              caixa        valor rendimento % rendimento
caixa_investimentos  R$ 8,375.55       % 3.15  R$ 263.94
        CDB DI Itaú R$ 32,781.00       % 0.00    R$ 0.00

ativo        valor valorização proventos
TAEE4  R$ 4,624.00     % -1.11   R$ 0.00
TAEE3  R$ 3,447.00    % -13.15 R$ 230.56
SUZB3  R$ 4,983.00      % 0.06   R$ 0.00
MGLU3    R$ 305.00    % -13.11   R$ 0.00
IRBR3  R$ 1,425.93     % 42.59   R$ 7.41
BBAS3  R$ 4,811.00     % 32.17 R$ 300.63
ITSA4 R$ 16,508.80      % 4.68 R$ 316.54
ITUB4  R$ 2,727.00      % 0.55  R$ 53.32
CMIG4  R$ 2,448.00      % 5.06  R$ 46.61
  MMM  R$ 1,975.00     % -3.49   R$ 0.00

    
    -------- Rodrigo Carteira --------

    Investido: R$49,535.56

    Valor atual: R$51,033.66
    Pro

In [5]:
print_report('dayane')


    -------- Dayane Patrimonio --------
    
    Patrimonio total: R$16,442.35

    Moeda Real: R$11,855.38
    Ativos Real: R$0.00
    Moeda Dolar: $435.03 (R$2,148.48)
    Ativos Dolar: $493.75 (R$2,438.48)

    ---------- Detalhamento ----------
    
        caixa        valor rendimento % rendimento
robank_dayane R$ 11,855.38       % 2.32  R$ 275.31

ativo     valor valorização proventos
  MMM R$ 493.75     % -3.49   R$ 0.00

    
    -------- Dayane Carteira --------

    Investido: R$2,526.64

    Valor atual: R$2,438.48
    Proventos: R$0.00
    Valor atual + Proventos: R$2,438.48

    Valorização: %-3.49
    Valorização + Proventos: %-3.49

    
